In [26]:
import os
from dotenv import load_dotenv
load_dotenv()
from datetime import timedelta,datetime
from openai import OpenAI
from langchain.messages import HumanMessage
import json

In [27]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get current time in HH:MM:SS format",
            "parameters": {"type": "object", "properties": {}},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_date",
            "description": "Get current date in YYYY-MM-DD format",
            "parameters": {"type": "object", "properties": {}},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_datetime",
            "description": "Get current date and time in YYYY-MM-DD HH:MM:SS format",
            "parameters": {"type": "object", "properties": {}},
        },
    },
]


# Functions
def get_current_time():
    return datetime.now().strftime("%H:%M:%S")


def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")


def get_current_datetime():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


function_map = {
    "get_current_time": get_current_time,
    "get_current_date": get_current_date,
    "get_current_datetime": get_current_datetime,
}

In [28]:
client=OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

messages=[]
messages.append(
    {
        "role":"user",
        "content":"What is 102 days from today's date, tell me exactly in DD:MM:YY format , manually calculate it from today's date ."
    }
)

completion = client.chat.completions.create(
    model="openrouter/free",
    messages=messages,
    tools=tools
)

completion

ChatCompletion(id='gen-1770126498-cx2D884MAaXKcXnb0cHz', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content="I'll help you calculate the date that is 102 days from today. First, let me get today's date.\n", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_6e92e86942d44de893ac3589', function=Function(arguments='{}', name='get_current_date'), type='function', index=0)], reasoning="\nThe user wants me to calculate what date is 102 days from today's date, and they want it in DD:MM:YY format. They want me to manually calculate it from today's date.\n\nFirst, I need to get today's date using the get_current_date function. Then I'll need to add 102 days to that date and format it as DD:MM:YY.\n\nLet me start by getting today's date.", reasoning_details=[{'format': 'unknown', 'index': 0, 'type': 'reasoning.text', 'text': "\nThe user wants me to calc

## Extracting the tool_call

In [29]:
tool_call = completion.choices[0].message.tool_calls[0]
print(f"AI wants to call: {tool_call.function.name}")
print(f"With arguments: {tool_call.function.arguments}")

AI wants to call: get_current_date
With arguments: {}


## Executing the function

In [30]:
import json
args = json.loads(tool_call.function.arguments)

func_name=tool_call.function.name
func=function_map[func_name]
result=func(**args)

## Sending the function result back to the AI model

In [31]:
messages.append(completion.choices[0].message)
messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": result})

# Getting final answer
response2 = client.chat.completions.create(
    model="meta-llama/llama-3.3-70b-instruct:free",
    messages=messages,
    tools=tools
)

print("\n🤖 AI's final answer:")
print(response2.choices[0].message.content)


🤖 AI's final answer:
Today's date is 03-02-26. To calculate the date 102 days from now, we need to add 102 days to the current date.

102 days is equivalent to 3 months and 12 days. 

So, if we add 3 months to the current date (03-02-26), we get 03-05-26. 

Then, if we add 12 days to this date (03-05-26), we get 15-05-26.

Therefore, the date 102 days from today's date is 15-05-26.
